# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [19]:

import numpy as np
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb',
                        password='adb2020')


In [20]:
# 1. Znajdź listę wszystkich filmów o tej samej długości.
df = pd.read_sql(f"""
        select
        title,
        length

        from
        film

        where length
        in (select length
                from
                (select length,
                count(title)
                from film
                group by length
                having count(title) > 1) as sel)
                order by length asc
            """, connection)
df

,title,length
0,Ridgemont Submarine,46
1,Iron Moon,46
2,Alien Center,46
3,Kwai Homeward,46
4,Labyrinth League,46
...,...,...
995,Darn Forrester,185
996,Sweet Brotherhood,185
997,Soldiers Evolution,185
998,Worst Banger,185


In [21]:
df = pd.read_sql(f"""
        select
        title,
        length

        from
        film

        group by length, title
            """, connection)
df

,title,length
0,Independence Hotel,157
1,Crusade Honey,112
2,Birdcage Casper,103
3,Fool Mockingbird,158
4,Meet Chocolate,80
...,...,...
995,Date Speed,104
996,Airport Pollock,54
997,Unbreakable Karate,62
998,Godfather Diary,73


In [22]:
#2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

df = pd.read_sql(f"""
        select
        first_name,
        last_name,
        city

        from customer

        join address using (address_id)

        join city using (city_id)

        where city in (select city
            from (select count(*), city
                from
                customer

                join address using (address_id)
                join city using (city_id)
                group by city
                having count(*)>1) as citis)
                order by city
            """, connection)
df

,first_name,last_name,city
0,Scott,Shelley,Aurora
1,Clinton,Buford,Aurora
2,Mattie,Hoffman,London
3,Cecil,Vines,London


In [23]:
#3. Oblicz średni koszt wypożyczenia wszystkich filmów.

df = pd.read_sql(f"""
        select
        title,
        avg(payment.amount)

        from
        inventory

        join rental using (inventory_id)
        join payment using (rental_id)
        join film using (film_id)

        group by title
            """, connection)
df

,title,avg
0,Graceland Dynamite,6.323333
1,Opus Ice,6.090000
2,Braveheart Human,3.390000
3,Wonderful Drop,5.865000
4,Rush Goodfellas,2.918571
...,...,...
953,Mockingbird Hollywood,2.444545
954,Gathering Calendar,1.990000
955,Drums Dynamite,1.913077
956,Samurai Lion,4.240000


In [24]:
#4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

df = pd.read_sql(f"""
        select
        name,
        count(title)

        from
        category

        join film_category using (category_id)
        join film using (film_id)

        group by name
            """, connection)
df

,name,count
0,Sports,74
1,Classics,57
2,New,63
3,Family,69
4,Comedy,58
5,Animation,66
6,Travel,57
7,Music,51
8,Drama,62
9,Horror,56


In [25]:
#5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

df = pd.read_sql(f"""
        select
        country,
        count(*)

        from
        customer

        join address using (address_id)
        join city using (city_id)
        join country using (country_id)
        group by country
            """, connection)
df

,country,count
0,Bangladesh,3
1,Indonesia,14
2,Venezuela,7
3,Cameroon,2
4,Czech Republic,1
...,...,...
103,Tanzania,3
104,Poland,8
105,Greenland,1
106,French Guiana,1


In [26]:
#6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

df = pd.read_sql(f"""
        select
        store_id,
        manager_staff_id,
        count(customer_id)

        from
        customer

        join store using (store_id)
        group by store_id
        having count(customer_id) <= 300 and count(customer_id) >= 100
            """, connection)
df

,store_id,manager_staff_id,count
0,2,2,273


In [27]:
#7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

df = pd.read_sql(f"""
        select distinct
        c.first_name,
        c.last_name,
        sum(length)

        from
        film

        join inventory using (film_id)
        join rental using (inventory_id)
        join customer c using (customer_id)

        group by c.first_name, c.last_name
        having sum(length) >= 200
        order by sum(length),last_name asc
            """, connection)
df

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


In [28]:
#8. Oblicz średnią wartość wypożyczenia filmu.

df = pd.read_sql(f"""
        select
        title,
        avg(payment.amount)

        from
        inventory

        join rental using (inventory_id)
        join payment using (rental_id)
        join film using (film_id)
        group by title

            """, connection)
df

,title,avg
0,Graceland Dynamite,6.323333
1,Opus Ice,6.090000
2,Braveheart Human,3.390000
3,Wonderful Drop,5.865000
4,Rush Goodfellas,2.918571
...,...,...
953,Mockingbird Hollywood,2.444545
954,Gathering Calendar,1.990000
955,Drums Dynamite,1.913077
956,Samurai Lion,4.240000


In [29]:
#9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

df = pd.read_sql(f"""
        select
        c.name as category,
        round(avg(length), 5) as avg

        from film

        join film_category using (film_id)
        join category c using (category_id)
        group by c.name
            """, connection)
df

,category,avg
0,Sports,128.20270
1,Classics,111.66667
2,New,111.12698
3,Family,114.78261
4,Comedy,115.82759
5,Animation,111.01515
6,Travel,113.31579
7,Music,113.64706
8,Drama,120.83871
9,Horror,112.48214


In [30]:
#10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

df = pd.read_sql(f"""
        select
        name,
        title,
        length
        from
        film

        join film_category using (film_id)\
        join category c using (category_id)
        where(name, length)
        in(select
        c.name as category,
        max(length)
        from
        film

        join film_category using (film_id)
        join category c using (category_id)
        group by c.name)
        order by name
            """, connection)
df

,name,title,length
0,Action,Darn Forrester,185
1,Action,Worst Banger,185
2,Animation,Pond Seattle,185
3,Animation,Gangs Pride,185
4,Children,Fury Murder,178
5,Children,Wrong Behavior,178
6,Classics,Conspiracy Spirit,184
7,Comedy,Control Anthem,185
8,Documentary,Young Language,183
9,Documentary,Wife Turn,183


In [31]:
# 11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.
df = pd.read_sql(f"""
        select
        name,
        title,
        length

        from
        film

        join film_category using (film_id)
        join category c using (category_id)
        where(name, length)
        in(select
        c.name as category,
        max(length)
        from
        film
        join film_category using (film_id)
        join category c using (category_id)
        group by c.name)
        order by length desc limit 1
            """, connection)
df

,name,title,length
0,Games,Chicago North,185
